In [1]:
pip install pandas numpy matplotlib tensorflow


     |████████████████████████████████| 601.3MB 21kB/s  eta 0:00:0111   |█████                           | 94.3MB 12.9MB/s eta 0:00:401MB 13.4MB/s eta 0:00:35     |███████████████████████▍        | 439.4MB 35.2MB/s eta 0:00:05     |█████████████████████████████▌  | 554.5MB 21.9MB/s eta 0:00:03
     |████████████████████████████████| 5.8MB 103.1MB/s eta 0:00:01
     |████████████████████████████████| 24.5MB 98.2MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 74.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.5MB/s  eta 0:00:01
     |████████████████████████████████| 5.1MB 48.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 1.3MB/s  eta 0:00:01
     |████████████████████████████████| 5.4MB 84.9MB/s eta 0:00:01            | 1.0MB 84.9MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 88.6MB/s eta 0:00:01
     |████████████████████████████████| 296kB 93.9MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 30.7MB/s eta 0:0

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [5]:
df = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [6]:
# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

# Split into train and test sets
train_size = int(len(df) * 0.67)
test_size = len(df) - train_size
train, test = df[0:train_size, :], df[train_size:len(df), :]

# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


ValueError: could not convert string to float: 'Product_001'